In [5]:
def haha(a: str, b: int):
    print('a', a, 'b', b)

def trigger_function(status: bool = False, func_name: str = '', args: dict = {}) -> None:
    """
    Trigger the function in the backend
    For example: if status is True, and func_name is queries_db
    then run queries_db(*args)
    Args:
        func_name (str): _description_
        args (dict): _description_
    """

    if status:
        if func_name in globals() and callable(globals()[func_name]):
            if args:
                globals()[func_name](**args)
            else:
                globals()[func_name]()

# Calling trigger_function with haha function
trigger_function(True, 'haha', {
    'a' : 'a',
    'b' : 1
})


a a b 1


In [22]:
def build_filter_query(**kwargs):
    """
    This function will build the filter query for SQLite.

    Returns:
    The select query for SQLite. Must have all the columns in the schema.
    """
    
    range_value = {}
    
    query = 'SELECT * FROM laptop_detail\nWHERE 1=1\n'
    
    for key, value in kwargs.items():
        if value is None:
            continue
        if isinstance(value, list):
            if key == 'disk_type':
                condition = f"AND lower({key}) LIKE '%{value}%'\n"
                query += condition
            elif key == 'cpu':
                condition = 'AND ('
                for v in value:
                    or_condition = f"lower({key}) LIKE '%{v}%' OR "
                    condition += or_condition
                # Remove the last OR
                condition = condition[:-3] + ')\n'
                query += condition
            else:
                condition = f'{key} IN ('
                if isinstance(value[0], str):
                    for v in value:
                        condition += f"'{v}',"
                else: # float or int
                    for v in value:
                        condition += f"{v},"
                        
                # Remove the last comma and add a closing bracket
                condition = condition[:-1] + ')'
                query += 'AND ' + condition + '\n'
        elif isinstance(value, str):
            condition = f"AND {key} = '{value}'\n"
            query += condition
        elif isinstance(value, int) or isinstance(value, float):
            if 'start' in key or 'end' in key:
                real_key = key.replace('start_','').replace('end_', '')
                
                if real_key not in range_value:
                    range_value[real_key] = {}
                
                if 'start' in key:
                    range_value[real_key]['start'] = value
                else:
                    range_value[real_key]['end'] = value
            else:
                condition = f'AND {key} = {value}\n'
                query += condition
    
    # Add range value
    for key, val_range in range_value.items():
        start_val = val_range.get('start', None)
        end_val = val_range.get('end', None)
        
        if start_val and end_val:
            condition = f'AND {key} BETWEEN {start_val} AND {end_val}\n'
        elif start_val is None:
            condition = f'AND {key} <= {end_val}\n'
        elif end_val is None:
            condition = f'AND {key} >= {start_val}\n'
        else:
            continue
        
        query += condition
    
    return query
                

# Example usage:
filter_params = {
    "start_present_price": None,
    "end_present_price": None,
    "manufacturer": None,
    "laptop_type": "gaming",
    "cpu": None,
    "cpu_generation": None,
    "disk_type": None,
    "start_disk_size": None,
    "end_disk_size": None,
    "start_ram_gb": None,
    "end_ram_gb": None,
    "max_ram_slot": None,
    "screen_size": None,
    "start_screen_refresh_rate": None,
    "end_screen_refresh_rate": None,
    "gpu_type": None,
    "gpu_model": None,
    "start_weigh_kg": None,
    "end_weigh_kg": None
}



query= build_filter_query(**filter_params)
print(query)


SELECT * FROM laptop_detail
WHERE 1=1
AND laptop_type = 'gaming'

